# Facility Location with patterns

In this task we will solve the Facility Location Problem using patterns. 
We have the following data:

* a range $I$ containing the potential locations where a facility can be opened
* a range $J$ containing the customers
* a tuple $b$ containing the customer demands $b_{j} \in \mathbb{Z}_{+}$ for every $j \in J$
* a tuple $f$ containing the opening costs $f_{i} \in \mathbb{Z}_{+}$ for each location $i \in I$
* a tuple $C$ containing the capacities $C_{i} \in \mathbb{Z}_{+}$ for each location $i \in I$
* a dictionary $c$ containing the connection costs $c_{i, j}$ for connecting facitily $i \in I$ with customer $j \in J$
* a tuple $P$ containing the power set of $J$

Our task is to find a feasible assignment of customers to facilities of minimum total cost, such that:

* every customer is assigned to exactly one opened facility
* total demand of all customers assigned to a facility $i \in I$ is at most its capacity $C_{i}$

The binary variable $l_{i, p}$ equals to $1$ if the subset of customers $p \subseteq J$ is assigned to the facility $i \in I$, otherwise $l_{i, p}$ equals to $0$.

The cell below imports the necessary libraries. To create our model we need `Gurobi`.

In [6]:
from gurobipy import *
from itertools import chain, combinations

## Data

We have following data: <br/>
`I`: a list of potential locations where a facility can be opened <br/>
`J`: a list of customers <br/>
`b`: a tuple of customers demands <br/>
`f`: a tuple of opening costs <br/>
`C`: a tuple of facility capacities <br/>
`c`: a dictionary of connection costs <br/>
`P`: a tuple of all patterns <br/>

In [7]:
I = range(3)

J = range(5)

b = (5, 7, 10, 14, 11)

f = (5, 11, 9)

C = (10, 20, 20)

c = {(0, 0): 2, (0, 1): 7, (0, 2): 7, (0, 3): 8, (0, 4): 8, (1, 0): 7,
  (1, 1): 2, (1, 2): 4, (1, 3): 7, (1, 4): 2, (2, 0): 2, (2, 1): 2,
  (2, 2): 2, (2, 3): 4, (2, 4): 8}

P = tuple(chain.from_iterable(combinations(J, r) for r in range(len(J)+1)))

#check
assert(len(J) == len(b))
assert(len(I) == len(f) == len(C))
assert(len(c) == len(I) * len(J))

## Model
Firstly, we need to create an empty model object `m` in `Gurobi` named `FacilityLocationWithPattern` where we will save all the details of our Facility Location With Pattern Model.<br/>For more details take a look at [Model()](https://www.gurobi.com/documentation/9.0/refman/py_model2.html).

In [16]:
m = Model('FacilityLocationWithPattern')

To decide whether we choose pattern $p$ of customers for location $i$ we introduce for every $i \in I$ and $p \in P$ a variable to our model `m`. <br/>For more details take a look at [addVar()](https://www.gurobi.com/documentation/9.0/refman/py_model_addvar.html).

In [17]:
l = {}
for i in I:
  for p in P:
    l[i, p] = m.addVar(vtype=GRB.BINARY, name=f'l[{i},{p}]')

For facility $i \in I$ let $\mathscr{P}^{i}$ be the set of all "patterns", i.e. $\mathscr{P}^{i}$ equals the power set of $J$. We define the costs $c_{P}^{i}$ of pattern $P \in \mathscr{P}^{i}$ for facility $i \in I$ by
$$c_{P}^{i} := \left\{\begin{array}{ll} f_{i} + \sum_{j \in P}{c_{ij}}, & \text{if } P \not = \emptyset \\
         0, & \text{if } P = \emptyset \end{array}\right. .$$ <br>
The cell below saves these costs in `cost`.


In [10]:
cost = {}
for i in I:
  for p in P:
    if len(p) == 0:
      cost[i, p] = 0
    else:
      cost[i, p] = f[i] + quicksum(c[i, j] for j in p)

Now, we can set our objective function to our model `m`. In this case, we create a minimizing problem and for every location $i$ and every pattern $p$, we multiply `l[i, p]` with its costs `cost[i, p]`. <br>
For more details take a look at [setObjective()](https://www.gurobi.com/documentation/9.0/refman/py_model_setobjective.html).

In [18]:
m.setObjective(quicksum(cost[i, p] * l[i, p] for i in I for p in P), GRB.MINIMIZE)

## Constraints
For facility $i \in I$, we exclude the non-feasible pattern, i.e. the subsets of customers that cannot be assigned to it at the same time. <br> 

In [19]:
for i in I:
  for p in P:
    if sum(b[j] for j in list(p)) > C[i]:
      m.addConstr(l[i, p] == 0)

For each facility $i \in I$, we choose exactly one pattern.

In [24]:
for i in I:
  m.addConstr(quicksum(l[i, p] for p in P) == 1)

For each customer $j \in J$, we choose exactly one pattern that assigns them to a facility.

In [25]:
for j in J:
  m.addConstr(quicksum(l[i, p] for i in I for p in P if j in list(p)) == 1)

In [26]:
m.update()
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 84 rows, 96 columns and 740 nonzeros
Model fingerprint: 0x6f5cfdbf
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 42

Presolve removed 84 rows and 96 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 42 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.200000000000e+01, best bound 4.200000000000e+01, gap 0.0000%
